In [4]:
D.<x> =PolynomialRing(QQ)
DD=PolynomialRing(QQ,'q')
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def extendedPolynomialCoefficient(n,poly): 
    pl=poly.list()
    if n<0:return 0
    if n>len(pl)-1:return 0
    return poly.list()[n]

def Delta(n):
    M = ModularForms(SL2Z, 12) 
    return DD(M.q_expansion_basis(n)[0])

def tau(n):
    return Delta(n+1)[n]
 
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})


print("okay")

okay


In [70]:
ls=[]
for n in [1..5]:ls=ls+[tau(n)]
print(ls)

[1, -24, 252, -1472, 4830]


In [6]:
import random

def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds


def RamanujanCongruenceSeries(n): 
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)

print(RamanujanCongruenceSeries(5))

-6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x


In [10]:
import random
def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds


def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)#+r*sm
        ans=ans+term*x^a
    return D(ans)


def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[k]

for k in [1..3]:
    print(RamanujanCongruenceSeries(k));print()
    print(recip(k)^k);print()
    rpc=recipPowerConstantTerm(k)
    test=order(2,rpc)-3*digitsum(2,k)
    print((rpc,test))
    
    print("----------------------------------------------------------")

-24*x^2 + x

x^-1 + 24 + 576*x + O(x^2)

(24, 0)
----------------------------------------------------------
252*x^3 - 24*x^2 + x

x^-2 + 48*x^-1 + 1224 + 19008*x + O(x^2)

(1224, 0)
----------------------------------------------------------
-1472*x^4 + 252*x^3 - 24*x^2 + x

x^-3 + 72*x^-2 + 2700*x^-1 + 70080 + 1427040*x + O(x^2)

(70080, 0)
----------------------------------------------------------


In [8]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[k]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

no=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..10]: # 100 < precision is necessary
        start=time.time()
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.03636789321899414)
(1, 2, 0, 0.06703996658325195)
(1, 3, 0, 0.07709169387817383)
(1, 4, 0, 0.08238911628723145)
(1, 5, 0, 0.09125018119812012)
(1, 6, 0, 0.12294983863830566)
(1, 7, 0, 0.12381100654602051)
(1, 8, 0, 0.13862872123718262)
(1, 9, 0, 0.13734817504882812)
(1, 10, 0, 0.12290716171264648)

bad a's for trial  1

[]
------------------------------------------------------------------
no:

[(1, 0)]


In [11]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[k]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..1]:
    print()
    ls=[]
    for a in [1..100]: # 100 < precision is necessary
        print((b,a))
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        tests=tests+[test]
    print("trial",b)
    print(tests)
    print("-----------------------------------------------------------------------")


(1, 1)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(1, 13)
(1, 14)
(1, 15)
(1, 16)
(1, 17)
(1, 18)
(1, 19)
(1, 20)
(1, 21)
(1, 22)
(1, 23)
(1, 24)
(1, 25)
(1, 26)
(1, 27)
(1, 28)
(1, 29)
(1, 30)
(1, 31)
(1, 32)
(1, 33)
(1, 34)
(1, 35)
(1, 36)
(1, 37)
(1, 38)
(1, 39)
(1, 40)
(1, 41)
(1, 42)
(1, 43)
(1, 44)
(1, 45)
(1, 46)
(1, 47)
(1, 48)
(1, 49)
(1, 50)
(1, 51)
(1, 52)
(1, 53)
(1, 54)
(1, 55)
(1, 56)
(1, 57)
(1, 58)
(1, 59)
(1, 60)
(1, 61)
(1, 62)
(1, 63)
(1, 64)
(1, 65)
(1, 66)
(1, 67)
(1, 68)
(1, 69)
(1, 70)
(1, 71)
(1, 72)
(1, 73)
(1, 74)
(1, 75)
(1, 76)
(1, 77)
(1, 78)
(1, 79)
(1, 80)
(1, 81)
(1, 82)
(1, 83)
(1, 84)
(1, 85)
(1, 86)
(1, 87)
(1, 88)
(1, 89)
(1, 90)
(1, 91)
(1, 92)
(1, 93)
(1, 94)
(1, 95)
(1, 96)
(1, 97)
(1, 98)
(1, 99)
(1, 100)
trial 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
import random
import time


def specialModulus(a):
    ds=digitsum(2,a)
    return 2*8^ds

def RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)

        term=tau(a)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(k+1)
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[k]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1010)

tests=[]
for b in [1..3]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        tests=tests+[test]
        print((b,a,test))
    print();print("trial",b)
    print();print(tests)
    print("-----------------------------------------------------------------------")


(1, 1, 0)
(1, 2, 0)
(1, 3, 0)
(1, 4, 0)
(1, 5, 0)
(1, 6, 0)
(1, 7, 0)
(1, 8, 0)
(1, 9, 0)
(1, 10, 0)
(1, 11, 0)
(1, 12, 0)
(1, 13, 0)
(1, 14, 0)
(1, 15, 0)
(1, 16, 0)
(1, 17, 0)
(1, 18, 0)
(1, 19, 0)
(1, 20, 0)
(1, 21, 0)
(1, 22, 0)
(1, 23, 0)
(1, 24, 0)
(1, 25, 0)
(1, 26, 0)
(1, 27, 0)
(1, 28, 0)
(1, 29, 0)
(1, 30, 0)
(1, 31, 0)
(1, 32, 0)
(1, 33, 0)
(1, 34, 0)
(1, 35, 0)
(1, 36, 0)
(1, 37, 0)
(1, 38, 0)
(1, 39, 0)
(1, 40, 0)
(1, 41, 0)
(1, 42, 0)
(1, 43, 0)
(1, 44, 0)
(1, 45, 0)
(1, 46, 0)
(1, 47, 0)
(1, 48, 0)
(1, 49, 0)
(1, 50, 0)
(1, 51, 0)
(1, 52, 0)
(1, 53, 0)
(1, 54, 0)
(1, 55, 0)
(1, 56, 0)
(1, 57, 0)
(1, 58, 0)
(1, 59, 0)
(1, 60, 0)
(1, 61, 0)
(1, 62, 0)
(1, 63, 0)
(1, 64, 0)
(1, 65, 0)
(1, 66, 0)
(1, 67, 0)
(1, 68, 0)
(1, 69, 0)
(1, 70, 0)
(1, 71, 0)
(1, 72, 0)
(1, 73, 0)
(1, 74, 0)
(1, 75, 0)
(1, 76, 0)
(1, 77, 0)
(1, 78, 0)
(1, 79, 0)
(1, 80, 0)
(1, 81, 0)
(1, 82, 0)
(1, 83, 0)
(1, 84, 0)
(1, 85, 0)
(1, 86, 0)
(1, 87, 0)
(1, 88, 0)
(1, 89, 0)
(1, 90, 0)
(1, 91, 0)
(1, 92,

(2, 67, 0)
(2, 68, 0)
(2, 69, 0)
(2, 70, 0)
(2, 71, 0)
(2, 72, 0)
(2, 73, 0)
(2, 74, 0)
(2, 75, 0)
(2, 76, 0)
(2, 77, 0)
(2, 78, 0)
(2, 79, 0)
(2, 80, 0)
(2, 81, 0)
(2, 82, 0)
(2, 83, 0)
(2, 84, 0)
(2, 85, 0)
(2, 86, 0)
(2, 87, 0)
(2, 88, 0)
(2, 89, 0)
(2, 90, 0)
(2, 91, 0)
(2, 92, 0)
(2, 93, 0)
(2, 94, 0)
(2, 95, 0)
(2, 96, 0)
(2, 97, 0)
(2, 98, 0)
(2, 99, 0)
(2, 100, 0)
(2, 101, 0)
(2, 102, 0)
(2, 103, 0)
(2, 104, 0)
(2, 105, 0)
(2, 106, 0)
(2, 107, 0)
(2, 108, 0)
(2, 109, 0)
(2, 110, 0)
(2, 111, 0)
(2, 112, 0)
(2, 113, 0)
(2, 114, 0)
(2, 115, 0)
(2, 116, 0)
(2, 117, 0)
(2, 118, 0)
(2, 119, 0)
(2, 120, 0)
(2, 121, 0)
(2, 122, 0)
(2, 123, 0)
(2, 124, 0)
(2, 125, 0)
(2, 126, 0)
(2, 127, 0)
(2, 128, 0)
(2, 129, 0)
(2, 130, 0)
(2, 131, 0)
(2, 132, 0)
(2, 133, 0)
(2, 134, 0)
(2, 135, 0)
(2, 136, 0)
(2, 137, 0)
(2, 138, 0)
(2, 139, 0)
(2, 140, 0)
(2, 141, 0)
(2, 142, 0)
(2, 143, 0)
(2, 144, 0)
(2, 145, 0)
(2, 146, 0)
(2, 147, 0)
(2, 148, 0)
(2, 149, 0)
(2, 150, 0)
(2, 151, 0)
(2, 152, 0)
(

(3, 5, 0)
(3, 6, 0)
(3, 7, 0)
(3, 8, 0)
(3, 9, 0)
(3, 10, 0)
(3, 11, 0)
(3, 12, 0)
(3, 13, 0)
(3, 14, 0)
(3, 15, 0)
(3, 16, 0)
(3, 17, 0)
(3, 18, 0)
(3, 19, 0)
(3, 20, 0)
(3, 21, 0)
(3, 22, 0)
(3, 23, 0)
(3, 24, 0)
(3, 25, 0)
(3, 26, 0)
(3, 27, 0)
(3, 28, 0)
(3, 29, 0)
(3, 30, 0)
(3, 31, 0)
(3, 32, 0)
(3, 33, 0)
(3, 34, 0)
(3, 35, 0)
(3, 36, 0)
(3, 37, 0)
(3, 38, 0)
(3, 39, 0)
(3, 40, 0)
(3, 41, 0)
(3, 42, 0)
(3, 43, 0)
(3, 44, 0)
(3, 45, 0)
(3, 46, 0)
(3, 47, 0)
(3, 48, 0)
(3, 49, 0)
(3, 50, 0)
(3, 51, 0)
(3, 52, 0)
(3, 53, 0)
(3, 54, 0)
(3, 55, 0)
(3, 56, 0)
(3, 57, 0)
(3, 58, 0)
(3, 59, 0)
(3, 60, 0)
(3, 61, 0)
(3, 62, 0)
(3, 63, 0)
(3, 64, 0)
(3, 65, 0)
(3, 66, 0)
(3, 67, 0)
(3, 68, 0)
(3, 69, 0)
(3, 70, 0)
(3, 71, 0)
(3, 72, 0)
(3, 73, 0)
(3, 74, 0)
(3, 75, 0)
(3, 76, 0)
(3, 77, 0)
(3, 78, 0)
(3, 79, 0)
(3, 80, 0)
(3, 81, 0)
(3, 82, 0)
(3, 83, 0)
(3, 84, 0)
(3, 85, 0)
(3, 86, 0)
(3, 87, 0)
(3, 88, 0)
(3, 89, 0)
(3, 90, 0)
(3, 91, 0)
(3, 92, 0)
(3, 93, 0)
(3, 94, 0)
(3, 95, 0)
(3, 